# Imports

In [9]:
import imutils
from imutils.video import FileVideoStream
import sys
import matplotlib.pyplot as plt
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')    
import os
import cv2
import time
import argparse
import multiprocessing
import numpy as np
import tensorflow as tf

sys.path.append('..')

from utils.app_utils import FPS, WebcamVideoStream
from multiprocessing import Queue, Pool
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Env setup

In [10]:
# This is needed to display the images.
%matplotlib inline

PATH_TO_MODEL = "../models/faster_rcnn_inception_resnet_v2_atrous_coco_2017_11_08/frozen_inference_graph.pb"
PATH_TO_LABELS = "../object_detection/data/mscoco_label_map.pbtxt"

NUM_CLASSES = 90

# Load label map

In [11]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES,
                                                           use_display_name = True)
category_index = label_map_util.create_category_index(categories)

# Detection sections ....

In [12]:
def detect_objects(image_np, sess, detection_graph):
    t_detect = time.time()
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Actual detection.
    (boxes, scores, classes, num_detections) = sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    t_elapsed = time.time() - t_detect
    print('[INFO] frame detection time: {:.2f}'.format(t_elapsed))
    
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=2)
    return image_np, t_elapsed

In [13]:
def worker(input_q, output_q):
    # Load a (frozen) Tensorflow model into memory.
    print("[INFO] loading TF model...")
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)
    print("[INFO] model loaded...")
    t_acc = 0
    while True:
        frame = input_q.get()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        im, t_elapsed = detect_objects(frame_rgb, sess, detection_graph)
        t_acc = t_acc + t_elapsed
        print('[INFO] accumulated detection time: {:.2f}'.format(t_acc))
        output_q.put(im)

    sess.close()

In [14]:
num_workers = 1
num_queue = 5

# debugging
#logger = multiprocessing.log_to_stderr()
#logger.setLevel(multiprocessing.SUBDEBUG)

input_q = Queue(maxsize=num_queue)
output_q = Queue(maxsize=num_queue)
pool = Pool(num_workers, worker, (input_q, output_q))

print("[INFO] starting video file thread...")
video_capture = FileVideoStream("../data/test_video.mp4").start()

time.sleep(1.0)
#video_capture = WebcamVideoStream(src=0,width=480,height=360).start()
fps = FPS().start()
framecnt = 0
while video_capture.more(): 
    #clear_output(wait=True)
    t = time.time()
    
    frame = video_capture.read()
    framecnt = framecnt + 1
    frame = imutils.resize(frame, width=480)
    input_q.put(frame)

    output_rgb = output_q.get()
    #output_rgb = cv2.cvtColor(output_q.get(), cv2.COLOR_RGB2BGR)
    #cv2.imshow('Video', output_rgb)
    #clear_output(wait=True)
    #plt.imshow(output_rgb)
    #plt.show()
    
    #if framecnt % 30 == 0:
    #    clear_output(wait=True)
    #    plt.figure(figsize=(20, 7))
    #    plt.imshow(output_rgb)
    #    plt.show()
    
    fps.update()
    
    print('[INFO] total frame elapsed time: {:.2f}'.format(time.time() - t))
    print('[INFO] queued frames: {:d}'.format(video_capture.Q.qsize()))

    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break
fps.stop()
print('[INFO] elapsed time (total): {:.2f}'.format(fps.elapsed()))
print('[INFO] approx. FPS: {:.2f}'.format(fps.fps()))
print('[INFO] total frames processed: {:d}'.format(framecnt))

pool.terminate()
video_capture.stop()
#cv2.destroyAllWindows()

[INFO] loading TF model...
[INFO] starting video file thread...
[INFO] model loaded...
[INFO] frame detection time: 78.39
[INFO] accumulated detection time: 78.39
[INFO] total frame elapsed time: 107.67
[INFO] queued frames: 128
[INFO] frame detection time: 58.57
[INFO] accumulated detection time: 136.96
[INFO] total frame elapsed time: 58.73
[INFO] queued frames: 128
[INFO] frame detection time: 58.99
[INFO] accumulated detection time: 195.95
[INFO] total frame elapsed time: 59.08
[INFO] queued frames: 128
[INFO] frame detection time: 59.27
[INFO] accumulated detection time: 255.23
[INFO] total frame elapsed time: 59.35
[INFO] queued frames: 128
[INFO] frame detection time: 59.73
[INFO] accumulated detection time: 314.96
[INFO] total frame elapsed time: 59.90
[INFO] queued frames: 128
[INFO] frame detection time: 57.69
[INFO] accumulated detection time: 372.65
[INFO] total frame elapsed time: 57.80
[INFO] queued frames: 128
[INFO] frame detection time: 59.02
[INFO] accumulated detecti

Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/Users/jesperchristensen/miniconda3/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 1302, in _run_fn
    status, run_metadata)
  File "/Users/jesperchristensen/miniconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/jesperchristensen/miniconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jesperchristensen/miniconda3/lib/python3.5/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "<ipython-input-13-e7297bc5ce22>", line 18, in worker
    im, t_elapsed = detect_objects(frame_rgb, sess, detection_graph)
  File "<ipython-input-12-c69ef983c421>", line 19, in detect_objects
    feed_dict={image_tensor: image_np_expanded})
  File "/Users/jesperchristensen/miniconda3/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 889, in run


KeyboardInterrupt: 

[INFO] loading TF model...
[INFO] model loaded...


# Summary

In [16]:
print('FPS for detection: {:.2f}'.format(framecnt/1032.73))
#print('FPS entire processing: {:.2f}'.format(912/fps.elapsed()))
#print('Time taken to post-process 31s videofile: {:.2f}s'.format(fps.elapsed()))

FPS for detection: 0.02
